Iris with tensorflow

In [17]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split

# 모델 정의
class IrisModel:
    def __init__(self) :
        # 가중치와 편향 초기화
        self.W1 = tf.Variable(tf.random.normal([4, 50]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([50]), dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal([50, 30]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([30]), dtype=tf.float32)
        self.W3 = tf.Variable(tf.random.normal([30, 3]), dtype=tf.float32)
        self.b3 = tf.Variable(tf.zeros([3]), dtype=tf.float32)
        
    def __call__(self, x):
        # 순전파 : 입력 -> 히든레이어 1 -> 히든레이어 2 -> 출력
        x = tf.cast(x, tf.float32)
        x = tf.nn.sigmoid(tf.matmul(x, self.W1) + self.b1)
        x = tf.nn.sigmoid(tf.matmul(x, self.W2) + self.b2)
        return tf.nn.softmax(tf.matmul(x, self.W3) + self.b3)
    
# 손실 함수 정의 (CrossEntropy)
def loss_fn(model, inputs, labels):
    labels = tf.cast(labels, tf.int32)
    predictions = model(inputs)
    labels_one_hot = tf.one_hot(labels, depth=3) # one-hot 인코딩
    loss = tf.reduce_mean(tf.losses.categorical_crossentropy(labels_one_hot, predictions))
    return loss

# 옵티마이저 설정 (Adam)
optimizer = tf.optimizers.Adam(learning_rate=0.001)

# 학습 함수 정의
def train_step(model, inputs, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, inputs, labels) # 손실 계산
    
    gradients = tape.gradient(loss, [model.W1, model.b1, model.W2, model.b2, model.W3, model.b3]) # 그래디언트 계산
    optimizer.apply_gradients(zip(gradients, [model.W1, model.b1, model.W2, model.b2, model.W3, model.b3])) # 가중치 업데이트
    return loss
    
# 정확도 계산 함수
def compute_accracy(model, inputs, labels):
    labels = tf.cast(labels, tf.int64)
    predictions = model(inputs)
    predicted_class = tf.argmax(predictions, axis=1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted_class, labels), tf.float32))
    return accuracy

# 데이터 로드
iris = datasets.load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# 데이터를 텐서로 변환
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.int64)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int64)


In [18]:
# 모델 초기화
model = IrisModel()

# 학습
num_epochs = 40
batch_size = 16
num_batches = int(np.ceil(len(X_train) / batch_size))

for epoch in range(num_epochs):
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        X_batch = X_train[start_idx:end_idx]
        y_batch = y_train[start_idx:end_idx]
        
        loss = train_step(model, X_batch, y_batch) # 학습 단계
        
    if epoch % 5 == 0: # 매 5 에포크마다 정확도 출력
        train_accuracy = compute_accracy(model, X_train, y_train)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss : {loss:.4f}, Accuracy : {train_accuracy:.4f}')
        

# 평가
test_accuracy = compute_accracy(model, X_test, y_test)
print(f'Test Accuracy : {test_accuracy:.4f}')

Epoch 1/40, Loss : 1.3442, Accuracy : 0.3524
Epoch 6/40, Loss : 0.9844, Accuracy : 0.5905
Epoch 11/40, Loss : 0.6092, Accuracy : 0.6190
Epoch 16/40, Loss : 0.4308, Accuracy : 0.9619
Epoch 21/40, Loss : 0.3579, Accuracy : 0.9619
Epoch 26/40, Loss : 0.3099, Accuracy : 0.9714
Epoch 31/40, Loss : 0.2830, Accuracy : 0.9714
Epoch 36/40, Loss : 0.2660, Accuracy : 0.9714
Test Accuracy : 0.9778
